<h1>Data Fitter</h1>

In [2]:
import os, sys
from importlib import reload

sys.path.append(os.getcwd() + '/../src/')

import XrayTools
reload(XrayTools)

xray = XrayTools.Fit()
xray.UI()

Box(children=(Dropdown(description='Parameter File', layout=Layout(width='70%'), options=('CO-O-Ru Hor 1-2 - E…

Dropdown(description='Data File', layout=Layout(width='50%'), options=('XAS_000_000_D1000_E100.h5', 'XAS_000_0…

Dropdown(description='Select Region', layout=Layout(width='40%'), options=('Pi Star', 'Middle', 'Shape Resonan…

Button(description='Fit', layout=Layout(width='10%'), style=ButtonStyle())

Output()